# Model class

In [1]:
import os

os.chdir("../..")

In [2]:
!pwd

/Users/wiktorlazarski/Desktop/my_project/head-segmentation


In [3]:
import head_segmentation.image_processing as ip
import head_segmentation.model as mdl

from scripts.training import augmentations as aug
from scripts.training import data_loading as dl

%load_ext autoreload
%autoreload 2

In [4]:
DATASET_PATH = os.path.join(os.getcwd(), "data", "dataset", "train")

aug_pipeline = aug.AugmentationPipeline()

dataset = dl.CelebAHeadSegmentationDataset(
    dataset_path=DATASET_PATH, 
    augmentation_pipeline=aug_pipeline,
    preprocess_pipeline=ip.PreprocessingPipeline(nn_image_input_resolution=256),
)

In [5]:
model = mdl.HeadSegmentationModel(encoder_name="mobilenet_v2", encoder_depth=3, pretrained=False, nn_image_input_resolution=256)

In [6]:
img, segmap = dataset[123]

In [7]:
type(segmap)

torch.Tensor

In [8]:
img.shape

torch.Size([3, 256, 256])

In [9]:
segmap.shape

torch.Size([256, 256])

In [10]:
segmap.dtype

torch.float32

In [11]:
segmap.max()

tensor(1.)

In [12]:
segmap.min()

tensor(0.)

In [13]:
mock_batch = img.unsqueeze(dim=0)

mdl_output = model.forward(mock_batch)

In [14]:
mdl_output.shape

torch.Size([1, 2, 256, 256])

In [15]:
# Model output to loss function
mdl_output

tensor([[[[ 0.1194,  0.2599, -1.2315,  ...,  0.9738,  0.9389,  0.3713],
          [ 0.1370,  0.1587,  0.2960,  ...,  2.2979,  1.5432,  1.1613],
          [ 0.7151,  0.1056,  0.0991,  ...,  2.2080,  0.0663,  1.3433],
          ...,
          [ 0.2272, -0.7556, -0.6755,  ...,  0.9743,  0.0851,  0.5221],
          [-0.2832, -0.8902, -1.0747,  ...,  0.8972, -0.1911,  0.8384],
          [ 0.4906,  0.2221, -0.4980,  ...,  0.0560, -0.0026,  0.6366]],

         [[-0.1192, -1.0621, -0.4571,  ..., -0.6474, -1.1605, -0.9606],
          [-0.1518, -1.8750, -2.1442,  ...,  0.4296,  0.2084, -0.5248],
          [-0.7634, -2.1603, -0.8940,  ...,  0.3653,  0.1224, -0.4869],
          ...,
          [-0.4403, -1.0304,  0.1246,  ...,  0.9251, -0.1813, -1.3981],
          [-0.0179, -1.6790, -0.5365,  ...,  0.7136, -0.1424, -0.8870],
          [-1.1235, -0.5468, -0.3840,  ..., -0.0487, -0.2084, -0.0397]]]],
       grad_fn=<ThnnConv2DBackward>)

In [16]:
def postprocessing(model_output):
    model_output = model_output.squeeze()
    segmap = model_output.argmax(dim=0)
    
    return segmap

In [17]:
mdl_output = postprocessing(mdl_output)

In [18]:
mdl_output.shape

torch.Size([256, 256])

In [19]:
mdl_output

tensor([[0, 0, 1,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 1, 0],
        ...,
        [0, 0, 1,  ..., 0, 0, 0],
        [1, 0, 1,  ..., 0, 1, 0],
        [0, 0, 1,  ..., 0, 0, 0]])

In [20]:
model

HeadSegmentationModel(
  (encoder): MobileNetV2Encoder(
    (features): Sequential(
      (0): ConvBNActivation(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6(inplace=True)
      )
      (1): InvertedResidual(
        (conv): Sequential(
          (0): ConvBNActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU6(inplace=True)
          )
          (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (2): InvertedResidual(
        (conv): Sequential(
          (0): ConvBNActivation(
            (0): Conv2d(16, 96, kernel_si